# 0. Install Dependencies

In [1]:
#pip install tensorflow==2.3.1 gym keras-rl2 gym[atari]

# 1. Test Random Environment with OpenAI Gym

In [15]:
import gym #to create our environment
import random #since we will be taking random steps in our env

### Create the gym environment

In [16]:
env = gym.make('SpaceInvaders-v0') #create gyme env
height, width, channels = env.observation_space.shape #
actions = env.action_space.n

### Actions

In [8]:
env.unwrapped.get_action_meanings()

['NOOP', 'FIRE', 'RIGHT', 'LEFT', 'RIGHTFIRE', 'LEFTFIRE']

### The game results before the agent is trained

In [17]:
episodes = 5
for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0 
    
    while not done:
        env.render()
        action = random.choice([0,1,2,3,4,5])
        n_state, reward, done, info = env.step(action)
        score+=reward
    print('Episode:{} Score:{}'.format(episode, score))
env.close()

Episode:1 Score:500.0
Episode:2 Score:110.0
Episode:3 Score:135.0
Episode:4 Score:125.0
Episode:5 Score:90.0


# 2. Create a Deep Learning Model with Keras

In [18]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Convolution2D
from tensorflow.keras.optimizers import Adam

### Building the model

In [19]:
def build_model(height, width, channels, actions):
    model = Sequential()
    model.add(Convolution2D(32, (8,8), strides=(4,4), activation='relu', input_shape=(3,height, width, channels)))
    model.add(Convolution2D(64, (4,4), strides=(2,2), activation='relu'))
    model.add(Convolution2D(64, (3,3), activation='relu'))
    model.add(Flatten())
    model.add(Dense(512, activation='relu')) 
    model.add(Dense(256, activation='relu'))
    model.add(Dense(actions, activation='linear'))
    return model

In [20]:
model = build_model(height, width, channels, actions)

Exception ignored in: <function SimpleImageViewer.__del__ at 0x0000028801FDCE50>
Traceback (most recent call last):
  File "C:\Users\hp\anaconda3\lib\site-packages\gym\envs\classic_control\rendering.py", line 382, in __del__
    self.close()
  File "C:\Users\hp\anaconda3\lib\site-packages\gym\envs\classic_control\rendering.py", line 378, in close
    self.window.close()
  File "C:\Users\hp\anaconda3\lib\site-packages\pyglet\window\win32\__init__.py", line 319, in close
    super(Win32Window, self).close()
  File "C:\Users\hp\anaconda3\lib\site-packages\pyglet\window\__init__.py", line 838, in close
    app.windows.remove(self)
  File "C:\Users\hp\anaconda3\lib\_weakrefset.py", line 109, in remove
    self.data.remove(ref(item))
KeyError: <weakref at 0x0000028801F46A90; to 'Win32Window' at 0x0000028801A1FF10>


In [9]:
del model

In [7]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 3, 51, 39, 32)     6176      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 3, 24, 18, 64)     32832     
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 3, 22, 16, 64)     36928     
_________________________________________________________________
flatten (Flatten)            (None, 67584)             0         
_________________________________________________________________
dense (Dense)                (None, 512)               34603520  
_________________________________________________________________
dense_1 (Dense)              (None, 256)               131328    
_________________________________________________________________
dense_2 (Dense)              (None, 6)                 1

# 3. Build Agent with Keras-RL

In [21]:
from rl.agents import DQNAgent
from rl.memory import SequentialMemory
from rl.policy import LinearAnnealedPolicy, EpsGreedyQPolicy

### Building the agent 

In [22]:
def build_agent(model, actions): #building agent using keras reinforcement learning
    policy = LinearAnnealedPolicy(EpsGreedyQPolicy(), attr='eps', value_max=1., value_min=.1, value_test=.2, nb_steps=10000)
    memory = SequentialMemory(limit=1000, window_length=3)
    dqn = DQNAgent(model=model, memory=memory, policy=policy,
                  enable_dueling_network=True, dueling_type='avg', 
                   nb_actions=actions, nb_steps_warmup=1000
                  )
    return dqn

In [23]:
dqn = build_agent(model, actions)
dqn.compile(Adam(lr=1e-4))

### defining the number of steps

In [ ]:
dqn.fit(env, nb_steps=10000, visualize=False, verbose=2)

In [28]:
scores = dqn.test(env, nb_episodes=5, visualize=True)
print(np.mean(scores.history['episode_reward']))

Testing for 5 episodes ...
Episode 1: reward: 470.000, steps: 1448
Episode 2: reward: 90.000, steps: 514
Episode 3: reward: 165.000, steps: 754
Episode 4: reward: 350.000, steps: 1001
Episode 5: reward: 250.000, steps: 726
265.0


# 4. Reloading Agent from Memory

In [28]:
dqn.save_weights('SavedWeights/10k-Fast/dqn_weights.h5f')

[WARNING] SavedWeights/10k-Fast/dqn_weights.h5f.index already exists - overwrite? [y/n]n


In [17]:
del model, dqn

In [27]:
dqn.load_weights('SavedWeights/1m/dqn_weights.h5f')